In [1]:
# データの読み込み
import pandas as pd
import numpy as np
from collections import Counter
import math
from scipy.optimize import minimize
import os
import glob
import openpyxl
import math
from scipy.optimize import minimize, fsolve
from scipy import optimize, exp
import scipy.optimize as optimize
from sympy import *
import sympy as sym
import time

# NumPyの表示オプションを設定して、整数表記にする
np.set_printoptions(suppress=True, precision=4)

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# ディレクトリ内のxlsxファイルのリストを取得 ###路線名変更
directory_path = '/Users/takuyamiyajima/Python/研究室/JFK-LAX'  # ディレクトリのパスを指定
xlsx_files = [f for f in os.listdir(directory_path) if f.endswith('.xlsx')]

# xlsxファイルを読み込んでDataFrameに格納し、リストに追加
dataframes = []
for xlsx_file in xlsx_files:
    file_path = os.path.join(directory_path, xlsx_file)
    df = pd.read_excel(file_path, index_col=0, engine="openpyxl")
    # 行名に"AA"を含む行のみを抽出 ###航空会社名変更
    df = df[df.index.astype(str).str.contains('AA', na=False)]
    dataframes.append(df)

# DataFrameを結合（インデックスをリセット）
df = pd.concat([df.reset_index(drop=True) for df in dataframes], axis=0) 

### クラス名変更
# Main Cabinの価格のみ取り出す
df = df.reset_index(drop=True)
def process_column(column):
    index_to_keep = []
    previous_row = None
    
    for index, cell in enumerate(column):
        if cell == 'Main Cabin':
            if previous_row is not None:
                index_to_keep.append(previous_row)
        previous_row = index
    
    return column.loc[index_to_keep]
df = df.apply(process_column)
# すべての列をfloat型に変換
df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

# DataFrameをNumPyの配列に変換
np_array = df.values

# 重複を除いた値を抽出してリストに格納する
unique_values = np.unique(np_array)
unique_values_list = unique_values.tolist()
# nanを削除
unique_values_list = [x for x in unique_values_list if not np.isnan(x)]
print(unique_values_list)

# 満期n日の場合
def pricing(data, target, gamma):
    # gamma = 0.01
    r = 1
    K = target * r

    # 初期推定値を設定
    initial_guess = 0.0  # 初期推定値
    lower_bound = 0
    upper_bound = 1
    # 制約条件を定義
    constraint = ({'type': 'ineq', 'fun': lambda beta: beta - lower_bound},  # beta >= lower_bound
                {'type': 'ineq', 'fun': lambda beta: upper_bound - beta})  # beta <= upper_bound

    # 最小化したい関数を定義
    def objective_function1(beta):
        term = 0
        for i in range(len(data)):
            if data[i,0] - target * r != 0:
                term += data[i,1] * np.exp( - (data[i,0] - target * r ) * gamma * beta )
            else:
                term += 0
        return term

    def objective_function2(beta):
        term = 0
        for i in range(len(data)):
            if data[i,0] != K:
                term += data[i,1] * np.exp( - ((data[i,0] - target * r ) * beta + max([data[i,0] - K, 0])) * gamma )
            else:
                term += 0
        return term

    # 最大化を実行
    result1 = minimize(objective_function1, initial_guess, constraints=constraint)
    # 最適な変数の値を表示
    beta1 = result1.x[0]
    # print(beta1)
    # print(result.fun)

    # 最大化を実行
    result2 = minimize(objective_function2, initial_guess, constraints=constraint)
    # 最適な変数の値を表示
    beta2 = result2.x[0]
    # print(beta2)
    # print(result.fun)

    x = Symbol('x')
    v = Symbol('v')

    A = 0
    for i in range(len(data)):
        A += - data[i,1] * exp( - ( r * x + ( data[i,0] - target * r ) * beta1 ) * gamma )
    
    B = 0
    for i in range(len(data)):
        B += - data[i,1] * exp( - ( r * (x-v) + ( data[i,0] - target * r ) * beta2 + max([data[i,0] - K, 0]) ) * gamma )

    solutions = solve(A-B, v)
    for sol in solutions:
        if sol.is_real:
            return sol

# 翌日のツリーを作成
def tree(target_value):
    if np.any(np_array == target_value):
        # 条件に一致する位置を取得
        indices = np.where(np_array == target_value)

        # 一致する位置の行と列をndarrayに格納
        positions = np.array(list(zip(indices[0], indices[1])))

        # 列番号を1足し、その場所の値をリストに格納
        values = []
        for row, col in positions:
            if col != np_array.shape[1] - 1:
                values.append(np_array[row, col + 1])

        # nanを削除
        values = [x for x in values if not np.isnan(x)]

        # 値の出現回数を数える
        value_counts = Counter(values)

        # 割合を計算し、値と割合を含むNumPyの2次元配列に格納
        result_array = np.array([(value, count / len(values) ) for value, count in value_counts.items()], dtype=float)

        # ソートしたい列のインデックスを指定（ここでは2列目をソートすると仮定）
        sort_column_index = 0

        try:
            # ソートのためのインデックスを取得
            sorted_indices = np.argsort(result_array[:, sort_column_index])[::-1]  # 降順にソート

            # 配列をソートしたインデックスに従って更新
            data = result_array[sorted_indices]

        except IndexError:
            # インデックスエラーが発生した場合、ここで処理をスキップ
            data = np.array([], dtype=int)  # 空のインデックス配列を作成

        return data
    
def pricing_for_next_day(data):
    array_list = []
    for i in range(len(data)):
        array_list.append(tree(data[i,0]))
    main_array = np.empty((0, 2))
    for j in range(len(array_list)):
        for k in range(len(array_list[j])):
            row = np.array([ array_list[j][k][0], data[j,1] * array_list[j][k][1]])
            main_array = np.vstack((main_array, row))
    # 結果を格納する新しいリストを作成
    result = []
    # ユニークな左の列の値を取得
    unique_left_column_values = np.unique(main_array[:, 0])
    # ユニークな左の列の値ごとに処理
    for value in unique_left_column_values:
        # 条件に合致する行を抽出
        rows_with_value = main_array[main_array[:, 0] == value]
        # 右の列の値を足し合わせて新しい行を作成し、結果リストに追加
        new_row = [value, np.sum(rows_with_value[:, 1])]
        result.append(new_row)
    # 結果をNumPy配列に変換
    result_array = np.array(result)
    # ソートしたい列のインデックスを指定（ここでは2列目をソートすると仮定）
    sort_column_index = 0
    # ソートのためのインデックスを取得
    sorted_indices = np.argsort(result_array[:, sort_column_index])[::-1]  # 降順にソート
    # 配列をソートしたインデックスに従って更新
    nextday = result_array[sorted_indices]
    # column_sum = np.sum(nextday[:, 1])  # 1番目の列を指定（0から始まるインデックスで1）
    # print("2列目の合計値:", column_sum)
    return nextday

def pricing_2day(x, gamma):
    one = tree(x)
    price1 = max([ pricing(one, x, gamma), 0 ])
    two = pricing_for_next_day(one)
    price2 = max([ pricing(two, x, gamma), price1 ])    
    return price2

def pricing_3day(x, gamma):
    one = tree(x)
    price1 = max([ pricing(one, x, gamma), 0 ])
    two = pricing_for_next_day(one)
    price2 = max([ pricing(two, x, gamma), price1 ])
    three = pricing_for_next_day(two)
    price3 = max ([ pricing(three, x, gamma), price2 ])
    return price3

def pricing_7day(x, gamma):
    one = tree(x)
    price1 = max([ pricing(one, x, gamma), 0 ])
    two = pricing_for_next_day(one)
    price2 = max([ pricing(two, x, gamma), price1 ])
    three = pricing_for_next_day(two)
    price3 = max ([ pricing(three, x, gamma), price2 ])
    four = pricing_for_next_day(three)
    price4 = max ([ pricing(four, x, gamma), price3 ])
    five = pricing_for_next_day(four)
    price5 = max ([ pricing(five, x, gamma), price4 ])
    six = pricing_for_next_day(five)
    price6 = max ([ pricing(six, x, gamma), price5 ])
    seven = pricing_for_next_day(six)
    price7 = max ([ pricing(seven, x, gamma), price6 ])
    return price7

def pricing_14day(x, gamma):
    one = tree(x)
    price1 = max([ pricing(one, x, gamma), 0 ])
    two = pricing_for_next_day(one)
    price2 = max([ pricing(two, x, gamma), price1 ])
    three = pricing_for_next_day(two)
    price3 = max ([ pricing(three, x, gamma), price2 ])
    four = pricing_for_next_day(three)
    price4 = max ([ pricing(four, x, gamma), price3 ])
    five = pricing_for_next_day(four)
    price5 = max ([ pricing(five, x, gamma), price4 ])
    six = pricing_for_next_day(five)
    price6 = max ([ pricing(six, x, gamma), price5 ])
    seven = pricing_for_next_day(six)
    price7 = max ([ pricing(seven, x, gamma), price6 ])
    eight = pricing_for_next_day(seven)
    price8 = max ([ pricing(eight, x, gamma), price7 ])
    nine = pricing_for_next_day(eight)
    price9 = max ([ pricing(nine, x, gamma), price8 ])
    ten = pricing_for_next_day(nine)
    price10 = max ([ pricing(ten, x, gamma), price9 ])
    eleven = pricing_for_next_day(ten)
    price11 = max ([ pricing(eleven, x, gamma), price10 ])
    twelve = pricing_for_next_day(eleven)
    price12 = max ([ pricing(twelve, x, gamma), price11 ])
    thirteen = pricing_for_next_day(twelve)
    price13 = max ([ pricing(thirteen, x, gamma), price12 ])
    fourteen = pricing_for_next_day(thirteen)
    price14 = max ([ pricing(fourteen, x, gamma), price13 ])
    return price14

hopper = pd.read_csv('Hopper_Price_Freeze.csv', index_col=0)
# hopper.iloc[45][0]

# width刻みの値を生成しリストに格納
# values1 = [2 * i * 10 ** (-3) for i in range(50, 5, -3)]

# values2 = [2 * i * 10 ** (-4) for i in range(50, 5, -3)]

# values3 = [2 * i * 10 ** (-5) for i in range(50, 5, -3)]

# values4 = [2 * i * 10 ** (-6) for i in range(50, 5, -3)]

values0 = [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]

values1 = [0.09,0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01]

values2 = [0.009,0.008,0.007,0.006,0.005,0.004,0.003,0.002,0.001]
# print(values1)
values3 = [0.0009,0.0006,0.0003]
# print(values2)
values4 = [0.00008,0.00007,0.00006]

values = values0 + values1 + values2 + values3 + values4

##停止位置
value_to_find = 959

if value_to_find in unique_values_list:
    index = unique_values_list.index(value_to_find)
    # print(f"値 {value_to_find} のインデックスは {index} です。")

# 出力用dataframe
df_price = pd.DataFrame(columns=['S_0', 'gamma2', 'price2', 'hopper2', 'gamma3', 'price3', 'hopper3','gamma7', 'price7', 'hopper7','gamma14', 'price14', 'hopper14'])
for x in unique_values_list[(index+1):]:
# for x in unique_values_list:
    print("S_0", x)
    hopper2 = hopper.iloc[int(x)][0]
    print(hopper2) 

    if pricing_2day(x, 1) - hopper2 < 0:
        temp1 = -1
    else:
        temp1 = 1
    print(1, pricing_2day(x, 1))
    if abs(pricing_2day(x, 1) - hopper2) < 10:
        gamma2_searched = 1
        price2_searched = pricing_2day(x, 1)
        print(gamma2_searched, price2_searched)
    else:
        if pricing_2day(x, 0.1) - hopper2 < 0:
            temp2 = -1
        else:
            temp2 = 1
        print(0.1, pricing_2day(x, 0.1))
        # if abs(pricing_2day(x, 0.1) - hopper2) < 10:
        #     gamma2_searched = 0.1
        #     price2_searched = pricing_2day(x, 0.1)
        #     print(gamma2_searched, price2_searched)
        # else:
    if temp1 * temp2 == -1:
        start_time = time.time()
        result_list = []
        diff = 10000000
        for gamma in values0:
            diff1 = diff
            price_searching = pricing_2day(x, gamma)
            diff = abs(hopper.iloc[int(x)][2] - price_searching)
            if diff > diff1:
                break
            result_list.append(price_searching)
            print(gamma, price_searching)
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time >= 120:
                print("処理が2分以上かかりました。")
                break
        gamma2_searched = values0[len(result_list) - 1]
        price2_searched = result_list[-1]
        print(gamma2_searched, price2_searched)
        
    else:
        if pricing_2day(x, 0.01) - hopper2 < 0:
            temp3 = -1
        else:
            temp3 = 1
        print(0.01, pricing_2day(x, 0.01))
        # if abs(pricing_2day(x, 0.01) - hopper2) < 10:
        #     gamma2_searched = 0.01
        #     price2_searched = pricing_2day(x, 0.01)
        #     print(gamma2_searched, price2_searched)

        if temp2 * temp3 == -1:
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values1:
                diff1 = diff
                price_searching = pricing_2day(x, gamma)
                diff = abs(hopper2 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma2_searched = values1[len(result_list) - 1]
            price2_searched = result_list[-1]
            print(gamma2_searched, price2_searched)
        
        elif abs(pricing_2day(x, 0.01) - hopper2) < 10:
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values2:
                diff1 = diff
                price_searching = pricing_2day(x, gamma)
                diff = abs(hopper2 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma2_searched = values2[len(result_list) - 1]
            price2_searched = result_list[-1]
            print(gamma2_searched, price2_searched)     

        else:
            if pricing_2day(x, 0.001) - hopper2 < 0:
                temp4 = -1
            else:
                temp4 = 1
            print(0.001, pricing_2day(x, 0.001))        
            # if abs(pricing_2day(x, 0.001) - hopper2) < 10:
            #     gamma2_searched = 0.001
            #     price2_searched = pricing_2day(x, 0.001)
            #     print(gamma2_searched, price2_searched)

            if temp3 * temp4 == -1:
                start_time = time.time()
                result_list = []
                diff = 10000000
                for gamma in values2:
                    diff1 = diff
                    price_searching = pricing_2day(x, gamma)
                    diff = abs(hopper2 - price_searching)
                    if diff > diff1:
                        break
                    result_list.append(price_searching)
                    print(gamma, price_searching)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    if elapsed_time >= 120:
                        print("処理が2分以上かかりました。")
                        break
                gamma2_searched = values2[len(result_list) - 1]
                price2_searched = result_list[-1]
                print(gamma2_searched, price2_searched)

            else:
                if pricing_2day(x, 0.0001) - hopper2 < 0:
                    temp5 = -1
                else:
                    temp5 = 1
                print(0.0001, pricing_2day(x, 0.0001))
                # if abs(pricing_2day(x, 0.0001) - hopper2) < 10:
                #     gamma2_searched = 0.0001
                #     price2_searched = pricing_2day(x, 0.0001)
                #     print(gamma2_searched, price2_searched)
                if temp4 * temp5 == -1:
                    start_time = time.time()
                    result_list = []
                    diff = 10000000
                    for gamma in values3:
                        diff1 = diff
                        price_searching = pricing_2day(x, gamma)
                        diff = abs(hopper2 - price_searching)
                        if diff > diff1:
                            break
                        result_list.append(price_searching)
                        print(gamma, price_searching)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        if elapsed_time >= 120:
                            print("処理が2分以上かかりました。")
                            break
                    gamma2_searched = values3[len(result_list) - 1]
                    price2_searched = result_list[-1]
                    print(gamma2_searched, price2_searched)
                else:
                    matrix = np.array([ [pricing_2day(x, 1), pricing_2day(x, 0.1), pricing_2day(x, 0.01), pricing_2day(x, 0.001), pricing_2day(x, 0.0001)], [1, 0.1, 0.01, 0.001, 0.0001] ] )
                    sabun = np.abs(matrix[0] - hopper2)
                    # 最も差が小さい要素のインデックスを取得
                    index_of_closest = np.argmin(sabun)
                    gamma2_searched = matrix[1, index_of_closest]
                    price2_searched = matrix[0, index_of_closest]
                    print(gamma2_searched, price2_searched)
    if abs(price2_searched - hopper2) > 30:
        continue

    hopper3 = hopper.iloc[int(x)][1]
    print(hopper3)

    if pricing_3day(x, 1) - hopper3 < 0:
        temp1 = -1
    else:
        temp1 = 1
    print(1, pricing_3day(x, 1))

    if pricing_3day(x, 0.1) - hopper3 < 0:
        temp2 = -1
    else:
        temp2 = 1
    print(0.1, pricing_3day(x, 0.1))

    if temp1 * temp2 == -1:
        start_time = time.time()
        result_list = []
        diff = 10000000
        for gamma in values0:
            diff1 = diff
            price_searching = pricing_3day(x, gamma)
            diff = abs(hopper3 - price_searching)
            if diff > diff1:
                break
            result_list.append(price_searching)
            print(gamma, price_searching)
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time >= 120:
                print("処理が2分以上かかりました。")
                break
        gamma3_searched = values0[len(result_list) - 1]
        price3_searched = result_list[-1]
        print(gamma3_searched, price3_searched)

    else:
        if pricing_3day(x, 0.01) - hopper3 < 0:
            temp3 = -1
        else:
            temp3 = 1
        print(0.01, pricing_3day(x, 0.01))

        if temp2 * temp3 == -1:
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values1:
                diff1 = diff
                price_searching = pricing_3day(x, gamma)
                diff = abs(hopper3 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma3_searched = values1[len(result_list) - 1]
            price3_searched = result_list[-1]
            print(gamma3_searched, price3_searched)
        
        elif abs(pricing_3day(x, 0.01) - hopper3) < 15:
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values2:
                diff1 = diff
                price_searching = pricing_3day(x, gamma)
                diff = abs(hopper3 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma3_searched = values2[len(result_list) - 1]
            price3_searched = result_list[-1]
            print(gamma3_searched, price3_searched)

        else:
            if pricing_3day(x, 0.001) - hopper3 < 0:
                temp4 = -1
            else:
                temp4 = 1
            print(0.001, pricing_3day(x, 0.001))
            if temp3 * temp4 == -1:
                start_time = time.time()
                result_list = []
                diff = 10000000
                for gamma in values2:
                    diff1 = diff
                    price_searching = pricing_3day(x, gamma)
                    diff = abs(hopper3 - price_searching)
                    if diff > diff1:
                        break
                    result_list.append(price_searching)
                    print(gamma, price_searching)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    if elapsed_time >= 120:
                        print("処理が2分以上かかりました。")
                        break
                gamma3_searched = values2[len(result_list) - 1]
                price3_searched = result_list[-1]
                print(gamma3_searched, price3_searched)
            
            elif abs(pricing_3day(x, 0.001) - hopper3) < 15:
                start_time = time.time()
                result_list = []
                diff = 10000000
                for gamma in values3:
                    diff1 = diff
                    price_searching = pricing_3day(x, gamma)
                    diff = abs(hopper3 - price_searching)
                    if diff > diff1:
                        break
                    result_list.append(price_searching)
                    print(gamma, price_searching)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    if elapsed_time >= 120:
                        print("処理が2分以上かかりました。")
                        break
                gamma3_searched = values3[len(result_list) - 1]
                price3_searched = result_list[-1]
                print(gamma3_searched, price3_searched)

            else:
                if pricing_3day(x, 0.0001) - hopper3 < 0:
                    temp5 = -1
                else:
                    temp5 = 1
                print(0.0001, pricing_3day(x, 0.0001))

                if temp4 * temp5 == -1:
                    start_time = time.time()
                    result_list = []
                    diff = 10000000
                    for gamma in values3:
                        diff1 = diff
                        price_searching = pricing_3day(x, gamma)
                        diff = abs(hopper3 - price_searching)
                        if diff > diff1:
                            break
                        result_list.append(price_searching)
                        print(gamma, price_searching)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        if elapsed_time >= 120:
                            print("処理が2分以上かかりました。")
                            break
                    gamma3_searched = values3[len(result_list) - 1]
                    price3_searched = result_list[-1]
                    print(gamma3_searched, price3_searched)

                elif abs(pricing_3day(x, 0.0001) - hopper3) < 15:
                    start_time = time.time()
                    result_list = []
                    diff = 10000000
                    for gamma in values4:
                        diff1 = diff
                        price_searching = pricing_3day(x, gamma)
                        diff = abs(hopper3 - price_searching)
                        if diff > diff1:
                            break
                        result_list.append(price_searching)
                        print(gamma, price_searching)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        if elapsed_time >= 120:
                            print("処理が2分以上かかりました。")
                            break
                    gamma3_searched = values4[len(result_list) - 1]
                    price3_searched = result_list[-1]
                    print(gamma3_searched, price3_searched)

                else:
                    matrix = np.array([ [pricing_3day(x, 1), pricing_3day(x, 0.1), pricing_3day(x, 0.01), pricing_3day(x, 0.001), pricing_3day(x, 0.0001)], [1, 0.1, 0.01, 0.001, 0.0001] ] )
                    sabun = np.abs(matrix[0] - hopper3)
                    # 最も差が小さい要素のインデックスを取得
                    index_of_closest = np.argmin(sabun)
                    gamma3_searched = matrix[1, index_of_closest]
                    price3_searched = matrix[0, index_of_closest]
                    print(gamma3_searched, price3_searched)

    if abs(price3_searched - hopper3) > 30:
        continue

    hopper7 = hopper.iloc[int(x)][2]
    print(hopper7) 

    if pricing_7day(x, 1) - hopper7 < 0:
        temp1 = -1
    else:
        temp1 = 1
    print(1, pricing_7day(x, 1))

    if pricing_7day(x, 0.1) - hopper7 < 0:
        temp2 = -1
    else:
        temp2 = 1
    print(0.1, pricing_7day(x, 0.1))
    if abs(pricing_7day(x, 0.1) - hopper7) < 10:
        gamma7_searched = 0.1
        price7_searched = pricing_7day(x, 0.1)
        print(gamma7_searched, price7_searched)

    elif temp1 * temp2 == -1:
        # pricing7(values0)
        start_time = time.time()
        result_list = []
        diff = 10000000
        for gamma in values0:
            diff1 = diff
            price_searching = pricing_7day(x, gamma)
            diff = abs(hopper7 - price_searching)
            if diff > diff1:
                break
            result_list.append(price_searching)
            print(gamma, price_searching)
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time >= 120:
                print("処理が2分以上かかりました。")
                break
        gamma7_searched = values0[len(result_list) - 1]
        price7_searched = result_list[-1]
        print(gamma7_searched, price7_searched)

    else:
        if pricing_7day(x, 0.01) - hopper7 < 0:
            temp3 = -1
        else:
            temp3 = 1
        print(0.01, pricing_7day(x, 0.01))
        if abs(pricing_7day(x, 0.01) - hopper7) < 10:
            gamma7_searched = 0.01
            price7_searched = pricing_7day(x, 0.01)
            print(gamma7_searched, price7_searched)

        elif temp2 * temp3 == -1:
            # pricing7(values1)
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values1:
                diff1 = diff
                price_searching = pricing_7day(x, gamma)
                diff = abs(hopper7 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma7_searched = values1[len(result_list) - 1]
            price7_searched = result_list[-1]
            print(gamma7_searched, price7_searched)

        elif abs(pricing_7day(x, 0.01) - hopper7) < 10:
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values2:
                diff1 = diff
                price_searching = pricing_7day(x, gamma)
                diff = abs(hopper7 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma7_searched = values2[len(result_list) - 1]
            price7_searched = result_list[-1]
            print(gamma7_searched, price7_searched)

        else:
            if pricing_7day(x, 0.001) - hopper7 < 0:
                temp4 = -1
            else:
                temp4 = 1
            print(0.001, pricing_7day(x, 0.001))
            if abs(pricing_7day(x, 0.001) - hopper7) < 10:
                gamma7_searched = 0.001
                price7_searched = pricing_7day(x, 0.001)
                print(gamma7_searched, price7_searched)

            elif temp3 * temp4 == -1:
                start_time = time.time()
                result_list = []
                diff = 10000000
                for gamma in values2:
                    diff1 = diff
                    price_searching = pricing_7day(x, gamma)
                    diff = abs(hopper7 - price_searching)
                    if diff > diff1:
                        break
                    result_list.append(price_searching)
                    print(gamma, price_searching)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    if elapsed_time >= 120:
                        print("処理が2分以上かかりました。")
                        break
                gamma7_searched = values2[len(result_list) - 1]
                price7_searched = result_list[-1]
                print(gamma7_searched, price7_searched)

            elif abs(pricing_7day(x, 0.001) - hopper7) < 20:
                start_time = time.time()
                result_list = []
                diff = 10000000
                for gamma in values3:
                    diff1 = diff
                    price_searching = pricing_7day(x, gamma)
                    diff = abs(hopper7 - price_searching)
                    if diff > diff1:
                        break
                    result_list.append(price_searching)
                    print(gamma, price_searching)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    if elapsed_time >= 120:
                        print("処理が2分以上かかりました。")
                        break
                gamma7_searched = values3[len(result_list) - 1]
                price7_searched = result_list[-1]
                print(gamma7_searched, price7_searched)

            else:
                if pricing_7day(x, 0.0001) - hopper7 < 0:
                    temp5 = -1
                else:
                    temp5 = 1
                print(0.0001, pricing_7day(x, 0.0001))
                if abs(pricing_7day(x, 0.0001) - hopper7) < 10:
                    gamma7_searched = 0.0001
                    price7_searched = pricing_7day(x, 0.0001)
                    print(gamma7_searched, price7_searched)
                elif temp4 * temp5 == -1:
                    start_time = time.time()
                    result_list = []
                    diff = 10000000
                    for gamma in values3:
                        diff1 = diff
                        price_searching = pricing_7day(x, gamma)
                        diff = abs(hopper7 - price_searching)
                        if diff > diff1:
                            break
                        result_list.append(price_searching)
                        print(gamma, price_searching)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        if elapsed_time >= 120:
                            print("処理が2分以上かかりました。")
                            break
                    gamma7_searched = values3[len(result_list) - 1]
                    price7_searched = result_list[-1]
                    print(gamma7_searched, price7_searched)

                elif abs(pricing_7day(x, 0.0001) - hopper7) < 20:
                    start_time = time.time()
                    result_list = []
                    diff = 10000000
                    for gamma in values4:
                        diff1 = diff
                        price_searching = pricing_7day(x, gamma)
                        diff = abs(hopper7 - price_searching)
                        if diff > diff1:
                            break
                        result_list.append(price_searching)
                        print(gamma, price_searching)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        if elapsed_time >= 120:
                            print("処理が2分以上かかりました。")
                            break
                    gamma7_searched = values4[len(result_list) - 1]
                    price7_searched = result_list[-1]
                    print(gamma7_searched, price7_searched)                    

                else:
                    matrix = np.array([ [pricing_7day(x, 1), pricing_7day(x, 0.1), pricing_7day(x, 0.01), pricing_7day(x, 0.001), pricing_7day(x, 0.0001)], [1, 0.1, 0.01, 0.001, 0.0001] ] )
                    sabun = np.abs(matrix[0] - hopper7)
                    # 最も差が小さい要素のインデックスを取得
                    index_of_closest = np.argmin(sabun)
                    gamma7_searched = matrix[1, index_of_closest]
                    price7_searched = matrix[0, index_of_closest]
                    print(gamma7_searched, price7_searched)
    if abs(price7_searched - hopper7) > 40:
        continue

    hopper14 = hopper.iloc[int(x)][3]
    print(hopper14)

    if pricing_14day(x, 1) - hopper14 < 0:
        temp1 = -1
    else:
        temp1 = 1
    print(1, pricing_14day(x, 1))

    if pricing_14day(x, 0.1) - hopper14 < 0:
        temp2 = -1
    else:
        temp2 = 1
    print(0.1, pricing_14day(x, 0.1))
    if abs(pricing_14day(x, 0.1) - hopper14) < 10:
        gamma14_searched = 0.1
        price14_searched = pricing_14day(x, 0.1)
        print(gamma14_searched, price14_searched)

    elif temp1 * temp2 == -1:
        # pricing7(values0)
        start_time = time.time()
        result_list = []
        diff = 10000000
        for gamma in values0:
            diff1 = diff
            price_searching = pricing_14day(x, gamma)
            diff = abs(hopper14 - price_searching)
            if diff > diff1:
                break
            result_list.append(price_searching)
            print(gamma, price_searching)
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time >= 120:
                print("処理が2分以上かかりました。")
                break
        gamma14_searched = values0[len(result_list) - 1]
        price14_searched = result_list[-1]
        print(gamma14_searched, price14_searched)

    else:
        if pricing_14day(x, 0.01) - hopper14 < 0:
            temp3 = -1
        else:
            temp3 = 1
        print(0.01, pricing_14day(x, 0.01))
        if abs(pricing_14day(x, 0.01) - hopper14) < 10:
            gamma14_searched = 0.01
            price14_searched = pricing_14day(x, 0.01)
            print(gamma14_searched, price14_searched)

        elif temp2 * temp3 == -1:
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values1:
                diff1 = diff
                price_searching = pricing_14day(x, gamma)
                diff = abs(hopper14 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma14_searched = values1[len(result_list) - 1]
            price14_searched = result_list[-1]
            print(gamma14_searched, price14_searched)

        elif abs(pricing_14day(x, 0.01) - hopper14) < 10:
            start_time = time.time()
            result_list = []
            diff = 10000000
            for gamma in values2:
                diff1 = diff
                price_searching = pricing_14day(x, gamma)
                diff = abs(hopper14 - price_searching)
                if diff > diff1:
                    break
                result_list.append(price_searching)
                print(gamma, price_searching)
                end_time = time.time()
                elapsed_time = end_time - start_time
                if elapsed_time >= 120:
                    print("処理が2分以上かかりました。")
                    break
            gamma14_searched = values2[len(result_list) - 1]
            price14_searched = result_list[-1]
            print(gamma14_searched, price14_searched)

        else:
            if pricing_14day(x, 0.001) - hopper14 < 0:
                temp4 = -1
            else:
                temp4 = 1
            print(0.001, pricing_14day(x, 0.001))
            if abs(pricing_14day(x, 0.001) - hopper14) < 10:
                gamma14_searched = 0.001
                price14_searched = pricing_14day(x, 0.001)
                print(gamma14_searched, price14_searched)

            elif temp3 * temp4 == -1:
                start_time = time.time()
                result_list = []
                diff = 10000000
                for gamma in values2:
                    diff1 = diff
                    price_searching = pricing_14day(x, gamma)
                    diff = abs(hopper14 - price_searching)
                    if diff > diff1:
                        break
                    result_list.append(price_searching)
                    print(gamma, price_searching)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    if elapsed_time >= 120:
                        print("処理が2分以上かかりました。")
                        break
                gamma14_searched = values2[len(result_list) - 1]
                price14_searched = result_list[-1]
                print(gamma14_searched, price14_searched)

            elif abs(pricing_14day(x, 0.001) - hopper14) < 30:
                start_time = time.time()
                result_list = []
                diff = 10000000
                for gamma in values3:
                    diff1 = diff
                    price_searching = pricing_14day(x, gamma)
                    diff = abs(hopper14 - price_searching)
                    if diff > diff1:
                        break
                    result_list.append(price_searching)
                    print(gamma, price_searching)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    if elapsed_time >= 120:
                        print("処理が2分以上かかりました。")
                        break
                gamma14_searched = values3[len(result_list) - 1]
                price14_searched = result_list[-1]
                print(gamma14_searched, price14_searched)

            else:
                if pricing_14day(x, 0.0001) - hopper14 < 0:
                    temp5 = -1
                else:
                    temp5 = 1
                print(0.0001, pricing_14day(x, 0.0001))
                if abs(pricing_14day(x, 0.0001) - hopper14) < 10:
                    gamma14_searched = 0.0001
                    price14_searched = pricing_14day(x, 0.0001)
                    print(gamma14_searched, price14_searched)

                elif temp4 * temp5 == -1:
                    start_time = time.time()
                    result_list = []
                    diff = 10000000
                    for gamma in values3:
                        diff1 = diff
                        price_searching = pricing_14day(x, gamma)
                        diff = abs(hopper14 - price_searching)
                        if diff > diff1:
                            break
                        result_list.append(price_searching)
                        print(gamma, price_searching)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        if elapsed_time >= 120:
                            print("処理が2分以上かかりました。")
                            break
                    gamma14_searched = values3[len(result_list) - 1]
                    price14_searched = result_list[-1]
                    print(gamma14_searched, price14_searched)

                elif abs(pricing_14day(x, 0.0001) - hopper14) < 30:
                    start_time = time.time()
                    result_list = []
                    diff = 10000000
                    for gamma in values4:
                        diff1 = diff
                        price_searching = pricing_14day(x, gamma)
                        diff = abs(hopper14 - price_searching)
                        if diff > diff1:
                            break
                        result_list.append(price_searching)
                        print(gamma, price_searching)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        if elapsed_time >= 120:
                            print("処理が2分以上かかりました。")
                            break
                    gamma14_searched = values4[len(result_list) - 1]
                    price14_searched = result_list[-1]
                    print(gamma14_searched, price14_searched)   

                else:
                    matrix = np.array([ [pricing_14day(x, 1), pricing_14day(x, 0.1), pricing_14day(x, 0.01), pricing_14day(x, 0.001), pricing_14day(x, 0.0001)], [1, 0.1, 0.01, 0.001, 0.0001] ] )
                    sabun = np.abs(matrix[0] - hopper14)
                    # 最も差が小さい要素のインデックスを取得
                    index_of_closest = np.argmin(sabun)
                    gamma14_searched = matrix[1, index_of_closest]
                    price14_searched = matrix[0, index_of_closest]
                    print(gamma14_searched, price14_searched)

    new_row = pd.DataFrame({'S_0': [x], 'gamma2': [gamma2_searched], 'price2': [price2_searched], 'hopper2': [hopper2], 'gamma3': [gamma3_searched], 'price3': [price3_searched], 'hopper3': [hopper3],'gamma7': [gamma7_searched], 'price7': [price7_searched], 'hopper7': [hopper7],'gamma14': [gamma14_searched], 'price14': [price14_searched], 'hopper14': [hopper14],})
    df_price = pd.concat([df_price, new_row], ignore_index=True)
    print(df_price)

In [2]:
df_price

,S_0,gamma2,price2,hopper2,gamma3,price3,hopper3,gamma7,price7,hopper7,gamma14,price14,hopper14
0,857.0,0.0060,38.4546719551500,38.0,0.0090,43.0408835878256,42.0,0.0100,59.3751390329944,68.0,0.0010,89.0183660406392,90.0
1,860.0,0.0001,28.0026556689935,38.0,0.0003,37.4782391137953,42.0,0.0009,56.8312803890128,68.0,0.0009,66.8998286125835,90.0
2,861.0,0.0030,38.9920960939830,38.0,0.0030,41.5762774917173,42.0,0.0020,67.3349738986999,68.0,0.0010,87.2105399566052,90.0
3,862.0,0.0100,42.5326259174607,38.0,0.0200,39.1319652211284,42.0,0.0100,71.7884588227556,68.0,0.0010,97.6145303511887,90.0
4,865.0,0.0700,33.2214764973910,38.0,0.0700,33.2214764973910,42.0,0.0100,59.8896132347206,68.0,0.0010,90.8042376002196,90.0
5,866.0,0.0090,38.2348598707642,38.0,0.0100,45.2642597648598,42.0,0.0100,63.4157583224332,68.0,0.0010,93.6782092877679,90.0
6,867.0,0.0070,37.7018062304662,38.0,0.0100,46.3475052969259,42.0,0.0100,69.7654700371608,68.0,0.0010,98.3418062612277,90.0
7,869.0,0.0060,38.5500929912649,38.0,0.0060,38.5500929912649,42.0,0.0040,73.7308035971960,68.0,0.0030,98.3910701828675,90.0


In [4]:
# 例！
import pandas as pd
import numpy as np
from collections import Counter
import math
from scipy.optimize import minimize
import os
import glob
import openpyxl
import math
from scipy.optimize import minimize, fsolve
from scipy import optimize, exp
import scipy.optimize as optimize
from sympy import *
gamma = 0.1
r = 1.1

data = np.array([[90, 1/3], [110, 1/3], [130, 1/3]])
print(data)

target = 100
K = target * r #(本当はtargetと等しい)

# 最小化したい関数を定義
def objective_function1(beta):
    term = 0
    for i in range(len(data)):
        if data[i,0] - target * r != 0:
            term += data[i,1] * np.exp( - (data[i,0] - target * r ) * gamma * beta )
        else:
            term += 0
    return term
    # return np.exp(2 * beta) + np.exp(-2 * beta)

# 初期推定値を設定
initial_guess = 0.0  # 初期推定値
# 最大化を実行
result = minimize(objective_function1, initial_guess, method='BFGS')
# 最適な変数の値を表示
beta1 = result.x[0]
print(beta1)

def objective_function2(beta):
    term = 0
    for i in range(len(data)):
        if data[i,0] != K:
            term += data[i,1] * np.exp( - ((data[i,0] - target * r ) * beta + max([data[i,0] - K, 0])) * gamma )
        else:
            term += 0
    return term
    # return math.exp(2 * beta) + math.exp(-2 * beta -2)

# 初期推定値を設定
initial_guess = 0.0  # 初期推定値
# 最大化を実行
result = minimize(objective_function2, initial_guess, method='BFGS')
# 最適な変数の値を表示
beta2 = result.x[0]
print(beta2)

x = Symbol('x')
v = Symbol('v')
gamma = 0.1

A = 0
for i in range(len(data)):
    A += - data[i,1] * exp( - ( r * x + ( data[i,0] - target * r ) * beta1 ) * gamma )
# A= - (exp(-(1.1 * x -20 * beta1)*gamma) + exp(-(1.1*x)*gamma) + exp(-(1.1*x + 20 * beta1)*gamma))/3
B = 0
for i in range(len(data)):
    B += - data[i,1] * exp( - ( r * (x-v) + ( data[i,0] - target * r ) * beta2 + max([data[i,0] - K, 0]) ) * gamma )
# B = - (exp(-(1.1 * (x-v) -20 * beta2)*gamma) + exp(-(1.1*(x-v))*gamma) + exp(-(1.1*(x-v) + 20 * beta2+20)*gamma))/3
# solve([x + 5*y*y - 2, -3*x + 6*y*y - 15], [x, y])
solutions = solve([A - B], [x, v])
# v の実数解を取得して格納
v_solutions = []

for sol in solutions:
    v_solution = sol[1]  # v はリスト内で2番目の要素（インデックス1）
    if v_solution.is_real:
        v_solutions.append(v_solution)

# v_solutions を出力
for v_solution in v_solutions:
    print(f"v_solution = {v_solution}")

[[ 90.     0.33]
 [110.     0.33]
 [130.     0.33]]
0.0
-0.5000000110869854
v_solution = 4.97425067941871


In [29]:
# 数式表示
import sympy 
from sympy import *
import numpy as np
# gamma = 0.1
# 初期推定値を設定
initial_guess = 0.0  # 初期推定値
lower_bound = -300
upper_bound = 300
# 制約条件を定義
constraint = ({'type': 'ineq', 'fun': lambda beta: beta - lower_bound},  # beta >= lower_bound
            {'type': 'ineq', 'fun': lambda beta: upper_bound - beta})  # beta <= upper_bound

data = np.array([[90, 0.1], [100, 0.2], [110, 0.3], [130, 0.4]])
# data = tree(224)
target = 110

# 最小化したい関数を定義
def objective_function1(beta):
    term = 0
    for i in range(len(data)):
        if data[i,0] - target != 0:
            term += gamma * data[i,1] * (( data[i,0] - target ) * beta)**gamma
        else:
            term += 0
    return term
    # return np.exp(2 * beta) + np.exp(-2 * beta)

def objective_function2(beta):
    term = 0
    for i in range(len(data)):
        if data[i,0] != target:
            term += gamma * data[i,1] * (( data[i,0] - target ) * beta + max([ data[i,0] - target , 0 ]))**gamma
        else:
            term += 0
    return term
    # return math.exp(2 * beta) + math.exp(-2 * beta -2)

# # 最大化を実行
# result = minimize(objective_function1, initial_guess, constraints=constraint)
# # 最適な変数の値を表示
# beta1 = result.x[0]
# # print(beta1)
# # print(result.fun)

# # 最大化を実行
# result = minimize(objective_function2, initial_guess, constraints=constraint)
# # 最適な変数の値を表示
# beta2 = result.x[0]
# # print(beta2)
# # print(result.fun)

x = sympy.Symbol('x')
v = sympy.Symbol('v')
beta1 = sympy.Symbol('beta1')
beta2 = sympy.Symbol('beta2')
gamma = sympy.Symbol('gamma')
r = 1
# gamma = 1
A = 0
for i in range(len(data)):
    A += - data[i,1] * exp( - ( r * x + ( data[i,0] - target ) * beta1 ) * gamma )

B = 0
for i in range(len(data)):
    B += - data[i,1] * exp( - ( r * (x-v) + ( data[i,0] - target ) * beta2 + max([data[i,0] - target, 0]) ) * gamma )

print(latex(simplify(A)))
print(latex(simplify(B)))
simplify(A)

sol = solve(A-B, v)
print(latex(sol[0]))
simplify(sol[0])

\left(- 0.3 e^{20.0 \beta_{1} \gamma} - 0.2 e^{30.0 \beta_{1} \gamma} - 0.1 e^{40.0 \beta_{1} \gamma} - 0.4\right) e^{- \gamma \left(20.0 \beta_{1} + x\right)}
\left(\left(- 0.3 e^{\gamma \left(v - x\right)} - 0.2 e^{\gamma \left(10.0 \beta_{2} + v - x\right)} - 0.1 e^{\gamma \left(20.0 \beta_{2} + v - x\right)}\right) e^{\gamma \left(20.0 \beta_{2} - v + x + 20.0\right)} - 0.4\right) e^{- \gamma \left(20.0 \beta_{2} - v + x + 20.0\right)}
\frac{\log{\left(\frac{\left(3.0 e^{20.0 \beta_{1} \gamma} + 2.0 e^{30.0 \beta_{1} \gamma} + e^{40.0 \beta_{1} \gamma} + 4.0\right) e^{20.0 \gamma \left(- \beta_{1} + \beta_{2} + 1.0\right)}}{3.0 e^{20.0 \gamma \left(\beta_{2} + 1.0\right)} + e^{20.0 \gamma \left(2.0 \beta_{2} + 1.0\right)} + 2.0 e^{10.0 \gamma \left(3.0 \beta_{2} + 2.0\right)} + 4.0} \right)}}{\gamma}


log((3.0*exp(20.0*beta1*gamma) + 2.0*exp(30.0*beta1*gamma) + exp(40.0*beta1*gamma) + 4.0)*exp(20.0*gamma*(-beta1 + beta2 + 1.0))/(3.0*exp(20.0*gamma*(beta2 + 1.0)) + exp(20.0*gamma*(2.0*beta2 + 1.0)) + 2.0*exp(10.0*gamma*(3.0*beta2 + 2.0)) + 4.0))/gamma